In [1]:
!pip install statsmodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 5.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.8/233.8 KB 7.1 MB/s eta 0:00:0031m12.0 MB/s eta 0:00:01


In [3]:
import numpy as np
from scipy.optimize import minimize

def arima(data, p, d, q):
    """
    Implements the ARIMA model on a time series using numpy and scipy.
    
    Parameters:
    - data: a 1-dimensional numpy array containing the time series data
    - p: the autoregressive (AR) order
    - d: the differencing order
    - q: the moving average (MA) order
    
    Returns:
    - predictions: a numpy array containing the forecasted values for the time series
    """
    
    # Define a function to calculate the residual sum of squares (RSS) for a given set of ARIMA coefficients
    def rss(params):
        ar_params = params[:p]
        ma_params = params[p:]
        y_hat = np.zeros_like(data)
        y_hat[:p] = data[:p]
        for i in range(p, len(data)):
            ar_term = np.sum(ar_params * y_hat[i-p:i])
            ma_term = np.sum(ma_params * (data[i-q:i] - y_hat[i-q:i]))
            y_hat[i] = ar_term + ma_term
        residuals = data - y_hat
        return np.sum(residuals**2)
    
    # Initialize ARIMA coefficients
    ar_coefs = np.zeros(p)
    ma_coefs = np.zeros(q)
    initial_guess = np.concatenate([ar_coefs, ma_coefs])
    
    # Minimize RSS to estimate coefficients
    result = minimize(rss, initial_guess, method='BFGS')
    ar_coefs = result.x[:p]
    ma_coefs = result.x[p:]
    
    # Apply the ARIMA model to the time series
    y_hat = np.zeros_like(data)
    y_hat[:p] = data[:p]
    for i in range(p, len(data)):
        ar_term = np.sum(ar_coefs * y_hat[i-p:i])
        ma_term = np.sum(ma_coefs * (data[i-q:i] - y_hat[i-q:i]))
        y_hat[i] = ar_term + ma_term
    
    # Apply differencing if necessary
    if d > 0:
        y_hat = np.cumsum(y_hat)
        y_hat = np.insert(y_hat, 0, data[0])
        y_hat = y_hat[:-d] - y_hat[d:]
    
    return y_hat

# Test the ARIMA function on sample data
data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
predictions = arima(data, p=1, d=1, q=1)
print("Predictions: ", predictions)

Predictions:  [0 0 0 0 0 0 0 0 0 0]


In [4]:
from statsmodels.tsa.arima_process import ArmaProcess

# Generate simulated data with known ARIMA parameters
ar_coefs = np.array([0.5, -0.2])
ma_coefs = np.array([0.3])
arima_process = ArmaProcess(ar_coefs, ma_coefs)
simulated_data = arima_process.generate_sample(nsample=100)

# Test the ARIMA function on simulated data
predictions = arima(simulated_data, p=2, d=0, q=1)

# Compare actual data and forecasted values
print("Actual data:    ", simulated_data[-10:])
print("Predictions:    ", predictions[-10:])
print("Prediction MSE: ", np.mean((simulated_data[-10:] - predictions[-10:])**2))


Actual data:     [-0.05771846  1.0977025   0.2530507   0.05193535 -0.5196793   0.56254967
  0.71386375 -0.07345681  0.30420713  0.43453438]
Predictions:     [ 0.05473348 -0.08886309  0.60811509 -0.12914269 -0.02312903 -0.2394722
  0.40959543  0.23117018 -0.21817672  0.21817847]
Prediction MSE:  0.2974316155271429
